### Import Libraries 

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, MobileNetV2, ResNet50
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


### Set Dataset Path

In [7]:
train_dir = r"E:\A\OneDrive\Documents\3rd year\5th SEM Materials of SR.Mishra(3rd yr)\Deep Learning for Image Analytics\Project\Groundnut_Leaf_dataset\Main Used Dataset"

# Updated image size
img_size = (224, 224)
batch_size = 32

NameError: name 'pd' is not defined

### Data Preprocessing

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data generator with normalization and validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# Training data generator
train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,     # Changed to (224, 224)
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

# Validation data generator
val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,     # Changed to (224, 224)
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

# Print number of classes
num_classes = len(train_gen.class_indices)
print("Number of classes:", num_classes)

Found 6329 images belonging to 6 classes.
Found 1581 images belonging to 6 classes.
Number of classes: 6


### Helper Function for Model Building

In [4]:
def build_model(base_model, preprocess_func):
    base_model.trainable = False
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=(224, 224, 3)),  # Updated from (128,128,3)
        tf.keras.layers.Lambda(preprocess_func),
        base_model,
        GlobalAveragePooling2D(),
        Dense(64, activation="relu"),
        Dropout(0.3),
        Dense(num_classes, activation="softmax")
    ])
    model.compile(
        optimizer=Adam(1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


### Train with ResNet50

In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',       # monitor validation loss
    patience=5,               # stop if no improvement for 5 epochs
    restore_best_weights=True # keep best model weights
)

# Build ResNet50 base model
resnet_base = ResNet50(
    weights="imagenet",       # use pretrained ImageNet weights
    include_top=False,        # exclude the fully connected layer
    input_shape=(128, 128, 3)
)

# Build full model using helper function
resnet_model = build_model(resnet_base, resnet_preprocess)

# Train the model
history_resnet = resnet_model.fit(
    train_gen,                # training generator
    validation_data=val_gen,  # validation generator
    epochs=50,
    callbacks=[early_stop]
)


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 196s 890ms/step - accuracy: 0.1898 - loss: 1.8038 - val_accuracy: 0.2650 - val_loss: 1.7669
Epoch 2/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 155s 782ms/step - accuracy: 0.2375 - loss: 1.7526 - val_accuracy: 0.2808 - val_loss: 1.7480
Epoch 3/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 133s 670ms/step - accuracy: 0.2842 - loss: 1.7250 - val_accuracy: 0.2815 - val_loss: 1.7321
Epoch 4/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 133s 675ms/step - accuracy: 0.3068 - loss: 1.6965 - val_accuracy: 0.3378 - val_loss: 1.7153
Epoch 5/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 133s 673ms/step - accuracy: 0.3291 - loss: 1.6754 - val_accuracy: 0.3017 - val_loss: 1.7019
Epoch 6/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 133s 672ms/step - accuracy: 0.3307 - loss: 1.6557 - val_accuracy: 0.3365 - val_loss: 1.6869
Epoch 7/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 134s 676ms/step - accuracy: 0.3505 - loss: 1.6336 - val_accuracy: 0.3080 - val_loss: 1.6747
Epoch 8/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 133s 670ms/step - accuracy: 0.3625 -

### Train with VGG19

In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',       # monitor validation loss
    patience=5,               # stop if no improvement for 5 epochs
    restore_best_weights=True # keep best model weights
)

# Build VGG19 base model
vgg_base = VGG19(
    weights="imagenet",       # use pretrained ImageNet weights
    include_top=False,        # exclude the fully connected layer
    input_shape=(128, 128, 3)
)

# Build full model using helper function
vgg_model = build_model(vgg_base, vgg_preprocess)

# Train the model
history_vgg = vgg_model.fit(
    train_gen,                # training generator
    validation_data=val_gen,  # validation generator
    epochs=50,
    callbacks=[early_stop]
)

Epoch 1/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 445s 2s/step - accuracy: 0.1722 - loss: 3.1474 - val_accuracy: 0.1885 - val_loss: 1.8220
Epoch 2/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 442s 2s/step - accuracy: 0.1694 - loss: 2.3559 - val_accuracy: 0.1891 - val_loss: 1.7890
Epoch 3/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 442s 2s/step - accuracy: 0.1681 - loss: 2.0872 - val_accuracy: 0.1898 - val_loss: 1.7832
Epoch 4/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 443s 2s/step - accuracy: 0.1691 - loss: 1.9357 - val_accuracy: 0.2037 - val_loss: 1.7804
Epoch 5/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 455s 2s/step - accuracy: 0.1760 - loss: 1.8618 - val_accuracy: 0.1910 - val_loss: 1.7785
Epoch 6/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 443s 2s/step - accuracy: 0.1970 - loss: 1.7971 - val_accuracy: 0.2043 - val_loss: 1.7751
Epoch 7/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 443s 2s/step - accuracy: 0.1883 - loss: 1.7868 - val_accuracy: 0.1841 - val_loss: 1.7732
Epoch 8/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 444s 2s/step - accuracy: 0.1958 - loss: 1.7738 - val_accu


### Train with MobileNetV2

In [5]:
# Early stopping to prevent overfitting
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Build MobileNetV2 base model with new input size
mobilenet_base = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)   # Updated from (128,128,3)
)

# Build complete model
mobilenet_model = build_model(mobilenet_base, mobilenet_preprocess)

# Train the model
history_mobilenet = mobilenet_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=50,
    callbacks=[early_stop]
)

C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 282s 1s/step - accuracy: 0.1861 - loss: 1.8235 - val_accuracy: 0.2113 - val_loss: 1.7795
Epoch 2/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 245s 1s/step - accuracy: 0.2079 - loss: 1.7719 - val_accuracy: 0.1879 - val_loss: 1.7781
Epoch 3/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 270s 1s/step - accuracy: 0.2274 - loss: 1.7565 - val_accuracy: 0.2106 - val_loss: 1.7701
Epoch 4/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.2501 - loss: 1.7419 - val_accuracy: 0.2625 - val_loss: 1.7598
Epoch 5/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 244s 1s/step - accuracy: 0.2651 - loss: 1.7253 - val_accuracy: 0.2612 - val_loss: 1.7524
Epoch 6/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 123s 620ms/step - accuracy: 0.2749 - loss: 1.7119 - val_accuracy: 0.2663 - val_loss: 1.7455
Epoch 7/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 83s 421ms/step - accuracy: 0.2836 - loss: 1.6914 - val_accuracy: 0.2479 - val_loss: 1.7421
Epoch 8/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 83s 418ms/step - accuracy: 0.2871 - loss: 1.6755 - v

In [14]:
import os

folder_path = r"E:\A\OneDrive\Documents\3rd year\5th SEM Materials of SR.Mishra(3rd yr)\Deep Learning for Image Analytics\Project\Groundnut_Leaf_dataset\Main Used Dataset"

print(os.listdir(folder_path))


['early_leaf_spot_1', 'early_rust_1', 'healthy_leaf_1', 'late_leaf_spot_1', 'nutrition_deficiency_1', 'rust_1']


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = r"E:\A\OneDrive\Documents\3rd year\5th SEM Materials of SR.Mishra(3rd yr)\Deep Learning for Image Analytics\Project\Groundnut_Leaf_dataset\Main Used Dataset"

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    subset='validation'
)


Found 6329 images belonging to 6 classes.
Found 1581 images belonging to 6 classes.


### Evaluation

In [9]:
# Evaluate all models on validation set
resnet_acc = resnet_model.evaluate(val_gen, verbose=0)[1]
vgg_acc = vgg_model.evaluate(val_gen, verbose=0)[1]
mobilenet_acc = mobilenet_model.evaluate(val_gen, verbose=0)[1]

print(f"ResNet50 Accuracy: {resnet_acc:.4f}")
print(f"VGG19 Accuracy: {vgg_acc:.4f}")
print(f"MobileNetV2 Accuracy: {mobilenet_acc:.4f}")

# Automatically determine best model
accuracies = {
    "ResNet50": resnet_acc,
    "VGG19": vgg_acc,
    "MobileNetV2": mobilenet_acc
}
best_model_name = max(accuracies, key=accuracies.get)
print("✅ Best Model:", best_model_name)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\A\\OneDrive\\Documents\\3rd year\\5th SEM Materials of SR.Mishra(3rd yr)\\Deep Learning for Image Analytics\\Project\\Groundnut_Leaf_dataset\\Main Used Dataset\\retail_data.csv'

### Compare Results

In [ ]:
plt.figure(figsize=(8,5))

# ResNet50
plt.plot(history_resnet.history['accuracy'], '--', label="ResNet50 Train")
plt.plot(history_resnet.history['val_accuracy'], label="ResNet50 Val")

# VGG19
plt.plot(history_vgg.history['accuracy'], '--', label="VGG19 Train")
plt.plot(history_vgg.history['val_accuracy'], label="VGG19 Val")

# MobileNetV2
plt.plot(history_mobilenet.history['accuracy'], '--', label="MobileNetV2 Train")
plt.plot(history_mobilenet.history['val_accuracy'], label="MobileNetV2 Val")

plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

### Save the Best Model

In [ ]:
# Automatically save the best model
if best_model_name == "ResNet50":
    resnet_model.save("best_groundnut_model.h5")
elif best_model_name == "VGG19":
    vgg_model.save("best_groundnut_model.h5")
else:
    mobilenet_model.save("best_groundnut_model.h5")

print("✅ Best model saved as 'best_groundnut_model.h5'")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, VGG19, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import ParameterGrid

# ===============================
#  Dataset Path
# ===============================
dataset_path = r"E:\A\OneDrive\Documents\3rd year\5th SEM Materials of SR.Mishra(3rd yr)\Deep Learning for Image Analytics\Project\Groundnut_Leaf_dataset\Main Used Dataset"

img_size = (224, 224)
batch_size = 32

# ===============================
#  Data Preprocessing
# ===============================
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

num_classes = len(train_gen.class_indices)

# ===============================
#  Model Builder Functions
# ===============================

def build_mobilenet_model(lr=1e-4, dropout_rate=0.3):
    base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=lr), loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_vgg19_model(lr=1e-4, dropout_rate=0.3):
    base_model = VGG19(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=lr), loss="categorical_crossentropy", metrics=["accuracy"])
    return model

def build_resnet50_model(lr=1e-4, dropout_rate=0.3):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=lr), loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# ===============================
#  Hyperparameter Grid
# ===============================
param_grid = {
    'lr': [1e-4, 5e-4],
    'dropout_rate': [0.3, 0.4],
    'batch_size': [16, 32]
}

# ===============================
#  Model Comparison
# ===============================
model_builders = {
    "MobileNetV2": build_mobilenet_model,
    "VGG19": build_vgg19_model,
    "ResNet50": build_resnet50_model
}

results = {}

for model_name, builder in model_builders.items():
    print(f"\n==============================")
    print(f"🚀 Training Model: {model_name}")
    print(f"==============================")

    best_acc = 0
    best_params = {}
    best_model = None

    for params in ParameterGrid(param_grid):
        print(f"\n🔍 Testing Params for {model_name}: {params}")
        model = builder(lr=params['lr'], dropout_rate=params['dropout_rate'])

        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=10,
            batch_size=params['batch_size'],
            callbacks=[EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)],
            verbose=1
        )

        val_acc = max(history.history['val_accuracy'])
        if val_acc > best_acc:
            best_acc = val_acc
            best_params = params
            best_model = model

    results[model_name] = {"accuracy": best_acc, "params": best_params}
    print(f"\n✅ Best for {model_name}: {best_params} with Accuracy: {best_acc:.4f}")

# ===============================
#  Compare and Print Final Results
# ===============================
print("\n==============================")
print("🏆 Final Model Comparison Results")
print("==============================")
for name, info in results.items():
    print(f"{name}: {info['accuracy']*100:.2f}% (Best Params: {info['params']})")

best_model_name = max(results, key=lambda x: results[x]['accuracy'])
print(f"\n🎯 Best Performing Model: {best_model_name} with Accuracy = {results[best_model_name]['accuracy']*100:.2f}%")

Found 6329 images belonging to 6 classes.
Found 1581 images belonging to 6 classes.

🚀 Training Model: MobileNetV2

🔍 Testing Params for MobileNetV2: {'batch_size': 16, 'dropout_rate': 0.3, 'lr': 0.0001}


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - accuracy: 0.3263 - loss: 1.7001 - val_accuracy: 0.4788 - val_loss: 1.3648
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 172s 871ms/step - accuracy: 0.5473 - loss: 1.1600 - val_accuracy: 0.6104 - val_loss: 1.0877
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 173s 872ms/step - accuracy: 0.6514 - loss: 0.9433 - val_accuracy: 0.6597 - val_loss: 0.9650
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 173s 872ms/step - accuracy: 0.6902 - loss: 0.8204 - val_accuracy: 0.6970 - val_loss: 0.8781
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 174s 879ms/step - accuracy: 0.7365 - loss: 0.7337 - val_accuracy: 0.7204 - val_loss: 0.8211
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 173s 876ms/step - accuracy: 0.7573 - loss: 0.6820 - val_accuracy: 0.7400 - val_loss: 0.7937
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 173s 876ms/step - accuracy: 0.7744 - loss: 0.6304 - val_accuracy: 0.7634 - val_loss: 0.7360
Epoch 8/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 173s 873ms/step - accuracy: 0.7870 - lo

198/198 ━━━━━━━━━━━━━━━━━━━━ 175s 882ms/step - accuracy: 0.7870 - loss: 0.5914 - val_accuracy: 0.7527 - val_loss: 0.7332
Epoch 9/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 173s 876ms/step - accuracy: 0.7925 - loss: 0.5748 - val_accuracy: 0.7672 - val_loss: 0.7081
Epoch 10/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 174s 881ms/step - accuracy: 0.8014 - loss: 0.5505 - val_accuracy: 0.7672 - val_loss: 0.6891

🔍 Testing Params for MobileNetV2: {'batch_size': 32, 'dropout_rate': 0.3, 'lr': 0.0005}
Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 181s 887ms/step - accuracy: 0.5906 - loss: 1.0688 - val_accuracy: 0.7432 - val_loss: 0.8158
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 172s 870ms/step - accuracy: 0.7748 - loss: 0.6189 - val_accuracy: 0.7862 - val_loss: 0.6808
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 172s 868ms/step - accuracy: 0.8128 - loss: 0.5130 - val_accuracy: 0.8065 - val_loss: 0.6337
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 172s 869ms/step - accuracy: 0.8368 - loss: 0.4555 - val_accuracy: 0.8216 - val_loss: 0.58